In [1]:
from detr_config import Config
from detr_factory import DETRFactory
from detr_file_manager import FileManager
from detr_model_loader import ModelLoader
from detr_data_source import DataSource
from detr_model_evaluator import ModelEvaluator
from detr_model import DETRModel
from detr_detection import plot_annotations, plot_results
from PIL import Image
import os
import torch

from transformers import (DeformableDetrConfig,
                          DeformableDetrForObjectDetection,
                          DeformableDetrImageProcessor, DetrConfig,
                          DetrForObjectDetection, DetrImageProcessor)


Config.set_local_settings()

In [2]:
hyperparameters = ('DETR', 25, 2)
detr_factory = DETRFactory(*hyperparameters)
file_manager = FileManager(detr_factory)
model_loader = ModelLoader(detr_factory, file_manager)
data_source = DataSource(detr_factory, file_manager)
file_manager.set_validation_setup(fold=1)

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [3]:
model = model_loader.load_best_model()

Loading Model:  lightning_logs\model=DETR_queries=25_layers=2\fold_1\checkpoints\epoch=51-step=312.ckpt
